In [1]:
%load_ext rich

# LLM Provider
A provider is a class that implements `ProviderProtocol`, serving as an interface to interact with a large language model (LLM). It processes user queries and returns model-generated responses.

```python
class ProviderProtocol(Protocol):
    async def achat(self, history: list[Message]) -> Message: ...
```

Expected Behavior

-  Implements `achat(self, history: list[Message]) -> Message`
-  Accepts a conversation history (list of `Message` objects)
-  Returns a single assistant-generated response (`Message`)

A provider facilitates structured communication with an LLM, ensuring consistency in message handling and response generation.

In [2]:
from promptimus import Message, MessageRole
from promptimus.llms import OpenAILike

In [3]:
# creating a provider
#
# do not forgent to export OPENAI_API_KEY with your token or pass it as api-key argument
provider = OpenAILike(
    model_name="gemma3:4b",
    base_url="http://lilan:11434/v1",
    api_key="DUMMY"
)

In [4]:
# using a provider with single message
await provider.achat(
    [
        Message(role=MessageRole.USER, content="Hi, who are you?"),
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="Hi there! I'm Gemma, a large language model created by the Gemma team at Google DeepMind. I'm an open-weights model, which means I'm widely available for public use! \n\nI can take text and images as inputs and output text. \n\nIt's nice to meet you!",
    tool_calls=None,
    tool_call_id=None
)

In [5]:
# using provider with multiple messages
await provider.achat(
    [
        Message(role=MessageRole.SYSTEM, content="Your name is Mark."),
        Message(role=MessageRole.USER, content="Hi, what is your name?"),
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='Hi there! You can call me Mark. It’s nice to meet you! 😊 \n\nWhat’s up with you?',
    tool_calls=None,
    tool_call_id=None
)

## OpenAI like and rate limits

In [6]:
from tqdm.auto import tqdm
import asyncio

In [7]:
openai = OpenAILike(
    model_name="gpt-4.1-nano",
    max_concurrency=50,
)

In [8]:
for task in tqdm.as_completed([openai.achat(
    [
        Message(role=MessageRole.USER, content="Hi, who are you?"),
    ]
) for _ in range(1_000)]):
    await task

  0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-23 14:43:26.180 | WARNING  | promptimus.llms.openai:achat:73 - Rate limit hit (attempt 1/5), waiting 3.000s (base 3.000s, exponential backoff)
2025-04-23 14:43:32.826 | INFO     | promptimus.llms.openai:achat:65 - Rate limit resolved.
2025-04-23 14:43:32.845 | WARNING  | promptimus.llms.openai:achat:73 - Rate limit hit (attempt 1/5), waiting 3.000s (base 3.000s, exponential backoff)
2025-04-23 14:43:33.359 | WARNING  | promptimus.llms.openai:achat:73 - Rate limit hit (attempt 2/5), waiting 6.000s (base 3.000s, exponential backoff)
2025-04-23 14:43:40.140 | WARNING  | promptimus.llms.openai:achat:73 - Rate limit hit (attempt 3/5), waiting 12.000s (base 3.000s, exponential backoff)
2025-04-23 14:43:52.978 | INFO     | promptimus.llms.openai:achat:65 - Rate limit resolved.
2025-04-23 14:43:54.583 | WARNING  | promptimus.llms.openai:achat:73 - Rate limit hit (attempt 4/5), waiting 24.000s (base 3.000s, exponential backoff)
2025-04-23 14:44:19.721 | INFO     | promptimus.llms.openai